In [2]:
import numpy as np
import pandas as pd

import scipy.stats 
import matplotlib.pyplot as plt
import seaborn as sns

# Налоговый период?

Ходят слухи, что когда наступает налоговый период, компании-экспортеры начинают резко продавать валютную выручку, создавая спрос на рубль. Таким образом, рубль должен укрепляться, те курс USDRUB должен идти вниз.

Почему это релевантно в отношении исследования Сургута? Мы посмотрим, имеет ли влияние налоговый период на котировки доллара, который уже непосредственно напрямую коррелирует с котировками Сургутнефтегаза. Корреляция есть, потому что это компания-экспортер и компания-держатель огромной валютной кубышки.

In [61]:
df_main = pd.read_csv('df_main.csv')
si = df_main[df_main['Код инструмента']=='Si']
sgns = df_main[df_main['Код инструмента']=='SNGS'] # подгрузили таблички с данными из предыдущего анализа

In [62]:
si

,Unnamed: 0,Дата торгов,Код инструмента,Срвзв.цена
0,0,2023-05-31,Si,80945.0
1,1,2023-05-30,Si,80730.0
2,2,2023-05-29,Si,80255.0
3,3,2023-05-26,Si,79889.0
4,4,2023-05-25,Si,79906.0
...,...,...,...,...
2016,2016,2015-06-05,Si,56478.0
2017,2017,2015-06-04,Si,55353.0
2018,2018,2015-06-03,Si,53795.0
2019,2019,2015-06-02,Si,53527.0


In [63]:
dates_taxes = sgns['Дата торгов'].to_frame()

dates_taxes['День'] = dates_taxes['Дата торгов'].apply(lambda x: int(x.replace('-', ""))%100)
dates_taxes #нашли день месяца, чтобы далее понять, относится ли он к налоговому периоду

,Дата торгов,День
6063,2023-05-31,31
6064,2023-05-30,30
6065,2023-05-29,29
6066,2023-05-26,26
6067,2023-05-25,25
...,...,...
8079,2015-06-05,5
8080,2015-06-04,4
8081,2015-06-03,3
8082,2015-06-02,2


In [64]:
#примем за условие, что валютная выручка в среднем продается 22-28 числа, и в этот момент рубль укрепляется

dates_taxes['Налоговый период'] = np.where(((dates_taxes['День']>=22) & (dates_taxes['День']<=28)), 1, 0)
siprice = si[['Срвзв.цена', 'Дата торгов']]
dt_tx_pr = dates_taxes.merge(siprice, how='left', on='Дата торгов')
sip = siprice['Срвзв.цена']
dates_taxes

,Дата торгов,День,Налоговый период
6063,2023-05-31,31,0
6064,2023-05-30,30,0
6065,2023-05-29,29,0
6066,2023-05-26,26,1
6067,2023-05-25,25,1
...,...,...,...
8079,2015-06-05,5,0
8080,2015-06-04,4,0
8081,2015-06-03,3,0
8082,2015-06-02,2,0


In [65]:
df_r = sip.diff() / sip.shift(1)
df_r = df_r.iloc[1:]
df_r = df_r.to_frame()

df_r['Дата торгов'] = np.array(dates_taxes['Дата торгов'][1:])
df_r['Динамика'] = np.where(df_r['Срвзв.цена']>=0, 0, 1) # единичка выходит, когда курс УСДРУБ идет вниз
df_r = df_r.rename(columns={"Срвзв.цена": "Процентное изменение"})
aggreg = df_r.merge(dates_taxes, on='Дата торгов', how='left')
hypotesis1 = aggreg[['Динамика', 'Налоговый период']]
hypotesis1 #вытащили признаки, которые нам нужны для конфьюжн матрикс, учитывая, что 1 и 1 - это условие роста рубля

,Динамика,Налоговый период
0,1,0
1,1,0
2,1,1
3,0,1
4,1,1
...,...,...
2015,0,0
2016,1,0
2017,1,0
2018,1,0


In [67]:
df_r # посмотрим, что получилось: процентное изменение, дата и полученный категориальный признак для проц изменения

,Процентное изменение,Дата торгов,Динамика
1,-0.002656,2023-05-30,1
2,-0.005884,2023-05-29,1
3,-0.004560,2023-05-26,1
4,0.000213,2023-05-25,0
5,-0.000813,2023-05-24,1
...,...,...,...
2016,0.004035,2015-06-05,0
2017,-0.019919,2015-06-04,1
2018,-0.028147,2015-06-03,1
2019,-0.004982,2015-06-02,1


__Для нашей гипотезы предположим, что если рубль укрепляется, то это 1. Если налоговый период, то также 1. То есть при 1 и 1 или 0 и 0 наша гипотеза верна. Построим таблицу.__

In [68]:
ct_o = hypotesis1.groupby('Налоговый период')['Динамика'].value_counts().unstack()
ct_o

Динамика,0,1
Налоговый период,,
0,801,745
1,228,246


In [70]:
accuracy = (246+801)/(246+801+745+228)

print(f'Accuracy = {accuracy}') #рандомайзер получается с нашими правильными ответами

Accuracy = 0.5183168316831683


Если посчитать отдельно Accuracy, получится, что наше предположение на уровне случайности. Посчитаем Хи-квадрат статистику, чтобы проверить наличие эффекта.

Н0: курс рубля и налоговый период не связаны

Н1: если начинается налоговый период, рубль укрепляется

In [75]:
rost_tax = 246/(246+228)
print (f'Доля дней, когда в налоговый период рубль действительно укреплялся: {rost_tax}')

Доля дней, когда в налоговый период рубль действительно укреплялся: 0.5189873417721519


In [76]:
a = scipy.stats.chi2_contingency(ct_o, correction=False)
chi2=a[0]
p_value=a[1]

print('chi2-статистика =', chi2)
print('p-value =', p_value)

if p_value < 0.05:
    print("Нулевая гипотеза отвергается. Эффект налогового периода наблюдается")
else:
    print("Нулевая гипотеза не отвергается. Эффекта налогового периода не наблюдается")

chi2-статистика = 1.9978603785503242
p-value = 0.1575214285513504
Нулевая гипотеза не отвергается. Эффекта налогового периода не наблюдается


# Вывод

Видим, что результаты очень неоднозначные ;) Это потому что у нас мало наблюдений, а еще потому что в нашей гипотезе неверно обратное: __если налогового периода нет, то вообще-то это совсем не значит, что рубль будет слабеть__. А вот у прямого утверждения есть теоретическое подкрепление, но для российского рынка оно не работает. Всегда есть факторы того, что макроэкономика неактуальна - очень много разных политических событий, а также штук, подконтрольных ЦБ, которые влияют на курс рубля и доллара. Однако если смотреть на горизонте пошире, ошибка будет уменьшаться.

__Поскольку четкой связи не обнаружено, мы приняли решение не интегрировать данный признак в дальнейшее машинное обучение.__